In [81]:
!pip install fake_useragent

## **Import Necessary Library**

In [82]:
from datetime import *
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from time import sleep, perf_counter
from threading import Thread
import pandas as pd
tqdm.pandas()

## **Manipulate Date&Time**


> This Block of Code When I started to Get the Current Time




In [83]:
#and the access its now method simpler
d = datetime.now().date()
d

datetime.date(2022, 4, 6)

## **The First Scrape**

> This Block Of Code When I started to Scrape urls,Number Of Nights,Check_in&Check_out

In [84]:
urls = []
xnights=[]
checkinlist=[]
checkoutlist=[]
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'
}


for i in range(2):
  checkin = d + timedelta(days=i)
  checkout = d + timedelta(days=i + 1)
  url='https://www.booking.com/searchresults.ar.html?src=searchresults&dest_id=-3091069&dest_type=city&checkin={}&checkout={}&nflt=ht_id%3D204&selected_currency=SAR&group_adults=1&ss=%D8%A7%D9%84%D8%AF%D9%85%D8%A7%D9%85&ssne=%D8%A7%D9%84%D8%AF%D9%85%D8%A7%D9%85&ssne_untouched=%D8%A7%D9%84%D8%AF%D9%85%D8%A7%D9%85&offset=0'.format(checkin,checkout)
  response=requests.get(url,headers=headers)
  soup=BeautifulSoup(response.content,'lxml')
  xnight=soup.find_all('div', {'data-testid':'price-for-x-nights'})
  titlelink=soup.find_all('a', {'data-testid':'title-link'})
  for night in xnight:
    xnights.append(night.text)
    checkinlist.append(checkin)
    checkoutlist.append(checkout)
  for link in titlelink : 
    links=link.get('href')
    urls.append(links)

In [87]:
print(urls)
print(checkinlist)
print(checkoutlist)
print(xnights)

['https://www.booking.com/hotel/sa/best-western-dammam.ar.html?aid=304142&ucfs=1&arphpl=1&checkin=2022-04-06&checkout=2022-04-07&dest_id=-3091069&dest_type=city&group_adults=1&req_adults=1&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&nflt=ht_id%3D204&srpvid=5cb74e61684f0150&srepoch=1649243332&all_sr_blocks=123375112_187974916_1_0_0&highlighted_blocks=123375112_187974916_1_0_0&matching_block_id=123375112_187974916_1_0_0&sr_pri_blocks=123375112_187974916_1_0_0__17000&from_beach_sr=1&from=searchresults#hotelTmpl', 'https://www.booking.com/hotel/sa/braira-dammam.ar.html?aid=304142&ucfs=1&arphpl=1&checkin=2022-04-06&checkout=2022-04-07&dest_id=-3091069&dest_type=city&group_adults=1&req_adults=1&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&nflt=ht_id%3D204&srpvid=5cb74e61684f0150&srepoch=1649243332&all_sr_blocks=337570401_187589741_2_0_0&highlighted_blocks=337570401_187589741_2_0_0&matching_block_id=337570401_187589741_2_0_0&s

## **DATA Length**

In [53]:
print(len(urls))
print(len(xnights))
print(len(checkinlist))
print(len(checkoutlist))

50
50
50
50


## **Generated DataFrame**


> I Created a Data Frame using the list in the first Scrape Which Contain 4 Columns  urls,nights,checkin&checkout



In [88]:
d={'URLS':urls ,'checkin':checkinlist,'checkout':checkoutlist,'nights':xnights}

In [89]:
import pandas as pd
df=pd.DataFrame(d)

In [90]:
df.head()

,URLS,checkin,checkout,nights
0,https://www.booking.com/hotel/sa/best-western-...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ
1,https://www.booking.com/hotel/sa/braira-dammam...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ
2,https://www.booking.com/hotel/sa/golden-tulip-...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ
3,https://www.booking.com/hotel/sa/wyndham-garde...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ
4,https://www.booking.com/hotel/sa/novotel-damma...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ


## **Main Method Helper 1|TITLES**


> I Created a Method Helper To Scrape The Titles For Each Hotel From The url  



In [57]:
def add_Titles(row):
  url = row["URLS"]
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'
}
  titles=[]
  try:
    session = requests.Session()
    retry = Retry(connect=10, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    session=session.get(url,headers=headers)
    soup=BeautifulSoup(session.content,'lxml')
    title=soup.find_all('h2', {'class':'hp__hotel-name'})
    for t in title:
      titles.append(t.text.replace('\n',''))
      return titles
  except IndexError:
    return 'null'

## **SubSidiary Method HELPER 1|Titles**

> I used a Method to Apply The titles Method  for Post Process

In [58]:
def data():
  print('Starting a task...')
  df["HotelNames"] = df.progress_apply(add_Titles, axis=1)
  print('done')

## **Main Method Helper 2|RoomType**

>I Created a Method Helper To Scrape The RoomType For Each Hotel From The url  


In [59]:
def add_RoomType(row):
  url = row["URLS"]
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'
}
  RoomTypelist=[]
  try:
    session = requests.Session()
    retry = Retry(connect=10, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    session=session.get(url,headers=headers)
    soup=BeautifulSoup(session.content,'lxml')
    RoomType=soup.find_all('span', {'class':'hprt-roomtype-icon-link '})

    for A in RoomType:
      RoomTypelist.append(A.text.replace('\n',''))
      return RoomTypelist
  except IndexError:
    return 'null'

## **SubSidiary Method HELPER 2|RoomType**

> I used a Method to Apply The RoomType Method  for Post Process

In [60]:
def RoomType():
  print('Starting a task...')
  df["RoomType"] = df.progress_apply(add_RoomType, axis=1)
  print('done')

## **Main Method Helper 3|Price**

>I Created a Method Helper To Scrape The Price For Each Hotel From The url  


In [61]:
def add_price(row):
  url = row["URLS"]
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'
}
  PriceList=[]
  try:
    session = requests.Session()
    retry = Retry(connect=10, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    session=session.get(url,headers=headers)
    soup=BeautifulSoup(session.content,'lxml')
    pricesoup=soup.find_all('span', {'class':'prco-valign-middle-helper'})
    for j in pricesoup:
      PriceList.append(j.text.replace('\n','').replace('\xa0',''))
      return PriceList
  except IndexError:
    return 'null'

## **SubSidiary Method HELPER 3|Price**

> I used a Method to Apply The Price Method  for Post Process

In [62]:
def price():
  print('Starting a task...')
  df["Price"] = df.progress_apply(add_price, axis=1)
  print('done')

## **Main Method Helper 4|Address**

>I Created a Method Helper To Scrape The Address For Each Hotel From The url  


In [63]:
def add_Address(row):
  url = row["URLS"]
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'
}
  AddressList=[]
  try:
    session = requests.Session()
    retry = Retry(connect=10, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    session=session.get(url,headers=headers)
    soup=BeautifulSoup(session.content,'lxml')
    Adress=soup.find_all('span', {'class':'hp_address_subtitle'})
    for ad in Adress:
      adress=ad.text
      AddressList.append(adress)
      return AddressList
  except IndexError:
    return 'null'

## **SubSidiary Method HELPER 4|Address**

> I used a Method to Apply The Address Method  for Post Process

In [64]:
def Address():
  print('Starting a task...')
  df["Address"] = df.progress_apply(add_Address, axis=1)
  print('done')

## **Main Method Helper 5|Location**

>I Created a Method Helper To Scrape The Location For Each Hotel From The url  


In [65]:
def add_location(row):
  url = row["URLS"]
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'
}
  latlon=[]
  try:
    session = requests.Session()
    retry = Retry(connect=10, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    session=session.get(url,headers=headers)
    soup=BeautifulSoup(session.content,'lxml')
    latclass=soup.find_all('a',class_='jq_tooltip')
    for m in latclass:
      lating=m.get('data-atlas-latlng')
      latlon.append(lating)
      return latlon[0]
  except IndexError:
    return 'null'

## **SubSidiary Method HELPER 5|Location**

> I used a Method to Apply The Location Method  for Post Process

In [66]:
def location():
  print('Starting a task...')
  df["latlon"] = df.progress_apply(add_location, axis=1)
  print('done')

## **Main Method Helper *6*|Rate**

>I Created a Method Helper To Scrape The Rate  For Each Hotel From The url  


In [67]:
def add_Rate(row):
  url = row["URLS"]
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'
}
  rating=[]

  try:
    session = requests.Session()
    retry = Retry(connect=10, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    session=session.get(url,headers=headers)
    soup=BeautifulSoup(session.content,'lxml')
    latrate=soup.find_all('span',class_='bui-rating')
    for m in latrate:
      Rating=m.get('aria-label')
      rating.append(Rating)
      return rating
  except IndexError:
    return 'null'

## **SubSidiary Method HELPER 6|Rate**

> I used a Method to Apply The Rate Method  for Post Process

In [68]:
def Rate():
  print('Starting a task...')
  df["Rating"] = df.progress_apply(add_Rate, axis=1)
  print('done')

## **Main Method HELPER 7|Score**

>I Created a Method Helper To Scrape The Score For Each Hotel From The url  


In [69]:
def add_Score(row):
  url = row["URLS"]
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'
}
  Scorelist=[]

  try:
    session = requests.Session()
    retry = Retry(connect=10, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    session=session.get(url,headers=headers)
    soup=BeautifulSoup(session.content,'lxml')
    Score=soup.find_all('span', {'class':'review-score-badge'})[1]
    for r in Score:
      Scores=r
      Scorelist.append(Scores)
      return Scorelist
  except IndexError:
    return 'null'

## **SubSidiary Method HELPER 7|Score**

> I used a Method to Apply The Score Method  for Post Process

In [70]:
def Score():
  print('Starting a task...')
  df["Score"] = df.progress_apply(add_Score, axis=1)
  print('done')

## **The Second Scrape**


> I Created 7 threads to Scrape Faster  




In [71]:
start_time = perf_counter()
# create a new threads
# start the threads

thread1 = Thread(target=data)
thread2 = Thread(target=RoomType)
thread3 = Thread(target=price)
thread4 = Thread(target=Address)
thread5 = Thread(target=location)
thread6 = Thread(target=Rate)
thread7 = Thread(target=Score)
# start the threads
thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread5.start()
thread6.start()
thread7.start()
thread1.join()
thread2.join()
thread3.join()
thread4.join()
thread5.join()
thread6.join()
thread7.join()
end_time = perf_counter()
print(f'It took {end_time- start_time: 0.2f} second(s) to complete.')

Starting a task...Starting a task...
Starting a task...





  0%|          | 0/50 [00:00<?, ?it/s]

Starting a task...
Starting a task...
Starting a task...





  0%|          | 0/50 [00:00<?, ?it/s]




  0%|          | 0/50 [00:00<?, ?it/s]



  0%|          | 0/50 [00:00<?, ?it/s]

Starting a task...


...

done



100%|██████████| 50/50 [03:55<00:00,  4.72s/it]


done



100%|██████████| 50/50 [04:02<00:00,  4.85s/it]

done
It took  242.41 second(s) to complete.


In [73]:
df.head()

,URLS,checkin,checkout,nights,HotelNames,latlon,Price,Score,Rating,Address,RoomType
0,https://www.booking.com/hotel/sa/best-western-...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,[فندقتوليب إن كورنيش الدمام],"26.45353733,50.13081222",[41US$],[\n6.4\n],[3 out of 5],[\nحي الشاطئ ، طريق الملك عبدالله بن عبد العزي...,[غرفة ديلوكس مزدوجة مطلة على البحر]
1,https://www.booking.com/hotel/sa/braira-dammam...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,[فندقBraira Al Dammam],"26.47016624,50.12401392",[67US$],[\n8.8\n],[4 out of 5],[\nBraira Dammam Hotel-Prince Mohammed Bin Fah...,[غرفة ديلوكس توأم]
2,https://www.booking.com/hotel/sa/sandan.ar.htm...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,[شقة فندقيةSendan Residence],"26.43583466,50.08783398",[53US$],[\n8.2\n],[4 out of 5],"[\nKing Khaled Street suite, 31311 الدمام, الم...",[شقة بغرفة نوم واحدة]
3,https://www.booking.com/hotel/sa/wryntl-rzydns...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,[فندقأورينتال رزيدنس الدمام],"26.36519098,50.09504381",[77US$],[\n8.4\n],[3 out of 5],[\nحي الفاخرية - ش. عمر بن الخطاب بجوار محطة ف...,[غرفة ديلوكس مزدوجة]
4,https://www.booking.com/hotel/sa/wyndham-garde...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,[فندقويندام جاردن الدمام],"26.42763200,50.09759400",[64US$],[\n8.4\n],[4 out of 5],"[\nشارع 21, 31411 الدمام, المملكة العربية السع...",[غرفة سوبيريور - لغير المدخنين]


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   URLS        50 non-null     object
 1   checkin     50 non-null     object
 2   checkout    50 non-null     object
 3   nights      50 non-null     object
 4   HotelNames  50 non-null     object
 5   Price       50 non-null     object
 6   Rating      41 non-null     object
 7   Address     50 non-null     object
 8   Score       50 non-null     object
 9   RoomType    50 non-null     object
 10  latlon      50 non-null     object
dtypes: object(11)
memory usage: 4.4+ KB


## **Convert List to String Mehod Helper**

In [74]:
#convert a list to string
    
# Function to convert  
def listToString(s): 
    
# initialize an empty string
    str1 = "" 
    
# traverse in the string  
    for ele in s: 
        str1 += ele  
    
# return string  
    return str1 

**Fill None value in Rating Column with 'unrated'**

In [75]:
df['Rating']=df['Rating'].fillna(value='Unrated')

**Apply the List to String method**

In [76]:
df['HotelNames']=df['HotelNames'].apply(listToString)
df['Price']=df['Price'].apply(listToString)
df['RoomType']=df['RoomType'].apply(listToString)
df['Address']=df['Address'].apply(listToString)
df['Rating']=df['Rating'].apply(listToString)
df['Score']=df['Score'].apply(listToString)

**Remove Unwanted text in a Dataframe**

In [77]:
import re
def clean_text(text):
  text=re.sub(r'US',' ',text)#Removing the 'US' word 
  text=re.sub(r'\n',' ',text)#Removing the '\n' symbol
  text=re.sub(r'out of 5',' ',text)#Removing the 'out of 5' sentence
  return text
df['Rating']=df['Rating'].apply(clean_text)
df['Address']=df['Address'].apply(clean_text)
df['Score']=df['Score'].apply(clean_text)
df['Price']=df['Price'].apply(clean_text)
df['Price'] = df['Price'].replace({'\$':''}, regex = True)#Removing the '$' symbol 

**Convert DataType and Multiply the price with 3.75**

In [79]:
df['Price'] = df['Price'].astype(int)#Convert object to string
df['Score'] = df['Score'].astype(float)
df['Price'] = df['Price'].apply(lambda x: x*3.75)#Multiply the Price by 3.75 to convert Dollar to Saudi Riyal

In [80]:
df

,URLS,checkin,checkout,nights,HotelNames,latlon,Price,Score,Rating,Address,RoomType
0,https://www.booking.com/hotel/sa/best-western-...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,فندقتوليب إن كورنيش الدمام,"26.45353733,50.13081222",153.75,6.4,3,حي الشاطئ ، طريق الملك عبدالله بن عبد العزيز ...,غرفة ديلوكس مزدوجة مطلة على البحر
1,https://www.booking.com/hotel/sa/braira-dammam...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,فندقBraira Al Dammam,"26.47016624,50.12401392",251.25,8.8,4,Braira Dammam Hotel-Prince Mohammed Bin Fahd ...,غرفة ديلوكس توأم
2,https://www.booking.com/hotel/sa/sandan.ar.htm...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,شقة فندقيةSendan Residence,"26.43583466,50.08783398",198.75,8.2,4,"King Khaled Street suite, 31311 الدمام, الممل...",شقة بغرفة نوم واحدة
3,https://www.booking.com/hotel/sa/wryntl-rzydns...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,فندقأورينتال رزيدنس الدمام,"26.36519098,50.09504381",288.75,8.4,3,حي الفاخرية - ش. عمر بن الخطاب بجوار محطة فيو...,غرفة ديلوكس مزدوجة
4,https://www.booking.com/hotel/sa/wyndham-garde...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,فندقويندام جاردن الدمام,"26.42763200,50.09759400",240.00,8.4,4,"شارع 21, 31411 الدمام, المملكة العربية السعودية",غرفة سوبيريور - لغير المدخنين
5,https://www.booking.com/hotel/sa/novotel-damma...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,فندقفندق نوفوتيل بزنس بارك الدمام,"26.40719760,50.13829708",296.25,7.8,4,"طريق الخبر الدمام السريع, 31413 الدمام, الممل...",غرفة قياسية مزدوجة
6,https://www.booking.com/hotel/sa/sheraton-damm...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,فندقفندق ومركز المؤتمرات شيراتون الدمام,"26.45184358,50.12025118",348.75,7.4,5,"شارع الوال ،الملك عبد العزيز الطريق, 55555 ا...",غرفة كلاسيكية، غرفة ضيوف في طابق منخفض مع سرير...
7,https://www.booking.com/hotel/sa/q-lgrf-l-zb.a...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,فندقQ Economy Rooms ألغرف ألأقتصاديه,"26.43977188,50.06007680",101.25,6.2,2,"عابد بن العوام, الدمام, 32248 الدمام, المملكة...",غرفة اقتصادية مزدوجة
8,https://www.booking.com/hotel/sa/tripper-inn.a...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,فندقTripper Inn,"26.48521220,50.12825980",213.75,8.9,4,"Yaqoot Street, 32411 الدمام, المملكة العربية ...",استوديو سوبيريور
9,https://www.booking.com/hotel/sa/park-inn-by-r...,2022-04-06,2022-04-07,ليلة واحدة، شخص واحد بالغ,فندقRadisson Hotel & Apartments Dammam Industr...,"26.23354000,49.98705100",300.00,7.9,4,"Dammam 2nd Industrial City, 67 Street, 2nd I...",غرفة قياسية
